### Joystick in ROS 2

ROS 2 has support for a variaty of joysticks through the `joy` package. This tutorial will go into the how the joystick works with ROS 2, while also practicing how to effectively use the command line. 

### Run the joystick
To test it out, make sure the joystick is connected to your computer through wire or bluetooth. To start up the joystick run 

```bash
ros2 run joy joy_node
```

in the terminal. If you succeeded you should see something like

```bash
[INFO] [1736755939.629722534] [joy_node]: Opened joystick: Wireless Controller.  deadzone: 0.050000
```
in your terminal window. What happened is that you started a node called `joy_node`. If you now run the 

```bash
ros2 node list
```

command to list all the active nodes, you should see `/joy_node`. If you want to see the subscribers and publishers of a specific node you can use the command 
```bash
ros2 node info "<node_name>"
```
So for the joy_node that would be
```bash
ros2 node info /joy_node
``` 
You should now see different topics and types being listed for the node. Most of this is stuff we don't care about. The interesting part is what the node outputs, i.e. the publisher(s). In this case you should see 
```bash
Publishers:
    /joy: sensor_msgs/msg/Joy
```
as part of the publishers. What this means is that data with type [Joy](https://docs.ros.org/en/noetic/api/sensor_msgs/html/msg/Joy.html) is published on topic `/joy`. To list the active topics and check if the `/joy` topic is there, run
```bash
ros2 topic list
```
You should now see the `/joy` topic listed. To check what data is being sent to the topic, we run the command
```bash
ros2 topic echo "<topic_name>"
```
which will be 
```bash
ros2 topic echo /joy
```
in our case. If done correctly, you should now see a stream of this message
```bash
header:
  stamp:
    sec: 1736755988
    nanosec: 430065583
  frame_id: joy
axes:
- -0.0
- -0.0
- 1.0
- -0.0
- -0.0
- 1.0
- 0.0
- 0.0
buttons:
- 0
- 0
- 0
- 0
- 0
- 0
- 0
- 0
- 0
- 0
- 0
- 0
- 0
```
What is interesting to look at here is the `axes` values and the `buttons` values. Now you can play around with the joystick, and see what buttons/sticks changes what values.

### How to use these values in your code

Consider this simple node

```python
import rclpy
from rclpy.node import Node
from sensor_msgs.msg import Joy

class JoystickNode(Node):
    def __init__(self):
        super().__init__('joystick_node')
        self.joy_subscriber = self.create_subscription(Joy, '/joy', self.joy_callback, 10)

    def joy_callback(self, msg: Joy):
        pass

def main(args=None):
    rclpy.init(args=args)
    node = JoystickNode()
    rclpy.spin(node)
    rclpy.shutdown()

if __name__ == '__main__':
    main()
```

#### Imports
The rclpy import lets us start up the node (spin). The Node import is like a template class which we want to inherit from, to gain access to for example the `create_subscription` method. This method allows us to create a subscriber to a topic. In our case, as we found out above, the topic name is `/joy` and the type is [Joy](https://docs.ros.org/en/noetic/api/sensor_msgs/html/msg/Joy.html). The `callback` is a method that will be called each time new messages are received on the topic. The callback method is where you decide what to do with the incoming message. The last part with the main function is just there to make sure that the node will run when the script is run.

#### Modify the joy_callback
Recall from the part where we used `echo` to see the content of the message. From there we see that the message has three fields: header, axes and buttons. In the [Joy documentation](https://docs.ros.org/en/noetic/api/sensor_msgs/html/msg/Joy.html) we see that axes and buttons are lists. If we want to access the list of axes and print it we can do
```python
def joy_callback(self, msg: Joy):
    axes = msg.axes
    print(axes)
```

This script can be used to play around with the joystick values. All you need to do is run the joy_node and the script. For your task in the lab, you need to identify which buttons and axes belongs to which index.